# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18169756508948691194
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11920893543
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2197203531905195993
physical_device_desc: "device: 0, name: TITAN Xp COLLECTORS EDITION, pci bus id: 0000:02:00.0, compute capability: 6.1"
]


In [4]:
# Add-on : Check framework versions being used 
import tensorflow as tf
import keras as K
import sys
print("Python", sys.version)
print("Tensorflow version", tf.__version__)
print("Keras version", K.__version__)

Python 3.5.5 |Anaconda, Inc.| (default, May 13 2018, 21:12:35) 
[GCC 7.2.0]
Tensorflow version 1.8.0
Keras version 2.2.0


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [5]:
# Remark : Encoding UTF-8 has been fixed in helper.py

# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [6]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [7]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [8]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'of': 14, 'quick': 2, 'by': 10, 'over': 7, 'short': 20, 'this': 18, 'fox': 5, 'prize': 17, 'lazy': 8, 'brown': 4, 'lexicography': 15, 'study': 13, 'is': 19, 'jove': 11, 'dog': 9, 'won': 16, 'sentence': 21, 'my': 12, 'a': 3, 'jumps': 6}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [9]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        # Find the length of the longest sequence/sentence
        length = max([len(seq) for seq in x])
    
    return pad_sequences(sequences=x, maxlen=length, padding='post')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [10]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [11]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [12]:
# Add-on : Collect logs for Tensorboard
from keras.callbacks import TensorBoard
from time import time
tensorboard = TensorBoard(log_dir="logs/{}".format(time()), histogram_freq=1, write_graph=True)

### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [13]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    
    input_seq = Input(shape=input_shape[1:])
    rnn = GRU(units=english_vocab_size, return_sequences=True)(input_seq)
    logits = TimeDistributed(Dense(units=french_vocab_size))(rnn) 
                             
    model = Model(input_seq, Activation('softmax')(logits))

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=1e-3),
                  metrics=['accuracy'])
    return model

tests.test_simple_model(simple_model)

# Pad and Reshape the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)

simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 4s 37us/step - loss: 2.7036 - acc: 0.4689 - val_loss: 2.0125 - val_acc: 0.5539
Epoch 2/10
110288/110288 [==============================] - 4s 32us/step - loss: 1.7538 - acc: 0.5741 - val_loss: 1.5904 - val_acc: 0.5918
Epoch 3/10
110288/110288 [==============================] - 4s 32us/step - loss: 1.5080 - acc: 0.6033 - val_loss: 1.4369 - val_acc: 0.6140
Epoch 4/10
110288/110288 [==============================] - 4s 32us/step - loss: 1.3838 - acc: 0.6230 - val_loss: 1.3332 - val_acc: 0.6305
Epoch 5/10
110288/110288 [==============================] - 4s 32us/step - loss: 1.2899 - acc: 0.6381 - val_loss: 1.2471 - val_acc: 0.6448
Epoch 6/10
110288/110288 [==============================] - 4s 32us/step - loss: 1.2145 - acc: 0.6494 - val_loss: 1.1827 - val_acc: 0.6538
Epoch 7/10
110288/110288 [==============================] - 4s 32us/step - loss: 1.1610 - acc: 0.6582 - val_loss: 1

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [14]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    #print("Debug input_shape =" , input_shape, "  Input length=", input_shape[1:][0])
    #print("Debug output_sequence_length =" , output_sequence_length) 
    #print("Debug english_vocab_size =" , english_vocab_size)
    #print("Debug french_vocab_size =" , french_vocab_size)
    
    # Hyperparameters
    embedding_size = 128
    rnn_cells = 200
    dropout = 0.0
    learning_rate = 1e-3
    
    # Sequential Model
    #from keras.models import Sequential
    #model = Sequential()
    #model.add(Embedding(english_vocab_size, embedding_size, input_length=input_shape[1:][0]))
    #model.add(GRU(rnn_cells, dropout=dropout, return_sequences=True))
    #model.add(Dense(french_vocab_size, activation='softmax'))
    #print(model.summary())
    
    # model's Functional equivalent
    input_seq = Input(shape=input_shape[1:])
     
    embedded_seq = Embedding(input_dim = english_vocab_size, 
                             output_dim = embedding_size,
                             input_length=input_shape[1:][0])(input_seq)
    
    rnn = GRU(units=rnn_cells, dropout=dropout, return_sequences=True)(embedded_seq)
    logits = TimeDistributed(Dense(units=french_vocab_size))(rnn) 
    model = Model(input_seq, Activation('softmax')(logits))
    #print(model.summary())

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])
    return model    
    
tests.test_embed_model(embed_model)


# Pad the input to work with the Embedding layer
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
#print("Debug tmp_x shape=", tmp_x.shape )


# Train the neural network 
embed_rnn_model = embed_model(input_shape = tmp_x.shape,
                              output_sequence_length = max_french_sequence_length,
                              english_vocab_size = english_vocab_size+1,
                              french_vocab_size = french_vocab_size+1)


embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 4s 39us/step - loss: 3.2682 - acc: 0.4139 - val_loss: 2.5310 - val_acc: 0.4774
Epoch 2/10
110288/110288 [==============================] - 4s 35us/step - loss: 2.0448 - acc: 0.5453 - val_loss: 1.5210 - val_acc: 0.6379
Epoch 3/10
110288/110288 [==============================] - 4s 35us/step - loss: 1.2809 - acc: 0.6886 - val_loss: 1.0707 - val_acc: 0.7419
Epoch 4/10
110288/110288 [==============================] - 4s 35us/step - loss: 0.9207 - acc: 0.7701 - val_loss: 0.7867 - val_acc: 0.7942
Epoch 5/10
110288/110288 [==============================] - 4s 34us/step - loss: 0.6843 - acc: 0.8152 - val_loss: 0.5990 - val_acc: 0.8331
Epoch 6/10
110288/110288 [==============================] - 4s 35us/step - loss: 0.5466 - acc: 0.8446 - val_loss: 0.4959 - val_acc: 0.8573
Epoch 7/10
110288/110288 [==============================] - 4s 35us/step - loss: 0.4616 - acc: 0.8660 - val_loss: 0

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [15]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    
    #print("Debug input_shape =" , input_shape)
    #print("Debug output_sequence_length =" , output_sequence_length) 
    #print("Debug english_vocab_size =" , english_vocab_size)
    #print("Debug french_vocab_size =" , french_vocab_size)
    
    # Hyperparameters
    dropout = 0.0
    learning_rate = 1e-3
    
    # Choose Sequential or Functional API implementation ('seq' or 'func')
    impl='seq'   
    if impl=='func':
        # Sequential Model 
        print("Using Sequential model (Note: this version makes the unitary test to fail: Disable tests to use it)")
        from keras.models import Sequential
        model = Sequential()
        model.add(Bidirectional(GRU(english_vocab_size, dropout=dropout, return_sequences=True)))
        model.add(Dense(french_vocab_size, activation='softmax'))
        
    else:
        # model's Functional equivalent
        # Note : we could have also used "Bidirectional(GRU(...))" instead of buidling the Bidirectional RNNS manually
        print("Using Functional API")
        from keras.layers import concatenate, add
        input_seq = Input(shape=input_shape[1:])
        right_rnn = GRU(units=english_vocab_size, return_sequences=True, go_backwards=False)(input_seq)
        left_rnn = GRU(units=english_vocab_size, return_sequences=True, go_backwards=True)(input_seq)

        # Choose how to merge the 2 rnn layers : add or concatenate
        #logits = TimeDistributed(Dense(units=french_vocab_size))(add([right_rnn, left_rnn])) 
        logits = TimeDistributed(Dense(units=french_vocab_size))(concatenate([right_rnn, left_rnn])) 
        
        model = Model(input_seq, Activation('softmax')(logits))

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])
  
    return model
   
tests.test_bd_model(bd_model)


# TODO: Train and Print prediction(s)

# Pad and Reshape the input to work with a RNN without an Embedding layer
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
#print("Debug tmp_x shape=", tmp_x.shape )

# Train the neural network 
bd_rnn_model = bd_model(input_shape = tmp_x.shape,
                           output_sequence_length = max_french_sequence_length,
                           english_vocab_size = english_vocab_size+1,
                           french_vocab_size = french_vocab_size+1)

#print(model.summary())

bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Using Functional API
Using Functional API
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 6s 55us/step - loss: 2.5609 - acc: 0.4781 - val_loss: 1.8599 - val_acc: 0.5583
Epoch 2/10
110288/110288 [==============================] - 5s 47us/step - loss: 1.6458 - acc: 0.5881 - val_loss: 1.5068 - val_acc: 0.6061
Epoch 3/10
110288/110288 [==============================] - 5s 47us/step - loss: 1.4290 - acc: 0.6161 - val_loss: 1.3564 - val_acc: 0.6268
Epoch 4/10
110288/110288 [==============================] - 5s 47us/step - loss: 1.3043 - acc: 0.6364 - val_loss: 1.2557 - val_acc: 0.6413
Epoch 5/10
110288/110288 [==============================] - 5s 47us/step - loss: 1.2172 - acc: 0.6492 - val_loss: 1.1803 - val_acc: 0.6534
Epoch 6/10
110288/110288 [==============================] - 5s 47us/step - loss: 1.1546 - acc: 0.6586 - val_loss: 1.1274 - val_acc: 0.6607
Epoch 7/10
110288/110288 [==============================] - 5s 47us/step 

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [16]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    
    #print("Debug input_shape =" , input_shape, "  input_shape[1:] =", input_shape[1:], "  Input length=", input_shape[1:][0])
    #print("Debug output_sequence_length =" , output_sequence_length) 
    #print("Debug english_vocab_size =" , english_vocab_size)
    #print("Debug french_vocab_size =" , french_vocab_size)
    
    # Hyperparameters
    embedding_size = 128
    rnn_cells = 200
    dropout = 0.0
    learning_rate = 1e-3
    
    from keras.layers import LSTM
    
    # Input
    encoder_input_seq = Input(shape=input_shape[1:], name="enc_input")
 
    # Encoder (Return the internal states of the RNN -> 1 hidden state for GRU cells, 2 hidden states for LSTM cells))
    encoder_output, state_t = GRU(units=rnn_cells, 
                                  dropout=dropout,
                                  return_sequences=False,
                                  return_state=True,
                                  name="enc_rnn")(encoder_input_seq)
          #or for LSTM cells: encoder_output, state_h, state_c = LSTM(...)
        
    # Decoder Input   
    decoder_input_seq = RepeatVector(output_sequence_length)(encoder_output)

    # Decoder RNN (Take the encoder returned states as initial states)
    decoder_out = GRU(units=rnn_cells,
                      dropout=dropout,
                      return_sequences=True,
                      return_state=False)(decoder_input_seq, initial_state=state_t)
                                         #or for LSTM cells: (decoder_input_seq, initial_state=[state_h, state_c])
    
    # Decoder output 
    logits = TimeDistributed(Dense(units=french_vocab_size))(decoder_out) 
    
    # Model
    model = Model(encoder_input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])
     
    return model    
    
# Unitary tests
tests.test_encdec_model(encdec_model)


# OPTIONAL: Train and Print prediction(s)

# Pad and Reshape the input to work with the Embedding layer
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
#print("Debug tmp_x shape=", tmp_x.shape )

# Train the neural network 
encdec_rnn_model = encdec_model(input_shape = tmp_x.shape,
                                output_sequence_length = max_french_sequence_length,
                                english_vocab_size = english_vocab_size+1,
                                french_vocab_size = french_vocab_size+1)
    
#print(encdec_rnn_model.summary())

encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2) # callbacks=[tensorboard]

# Print prediction(s)
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))
    
 

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 7s 62us/step - loss: 2.8793 - acc: 0.4567 - val_loss: 2.2250 - val_acc: 0.5084
Epoch 2/10
110288/110288 [==============================] - 6s 54us/step - loss: 2.0134 - acc: 0.5377 - val_loss: 1.7433 - val_acc: 0.5668
Epoch 3/10
110288/110288 [==============================] - 6s 53us/step - loss: 1.6446 - acc: 0.5719 - val_loss: 1.5747 - val_acc: 0.5792
Epoch 4/10
110288/110288 [==============================] - 6s 54us/step - loss: 1.5326 - acc: 0.5868 - val_loss: 1.5362 - val_acc: 0.5810
Epoch 5/10
110288/110288 [==============================] - 6s 54us/step - loss: 1.4378 - acc: 0.6055 - val_loss: 1.4000 - val_acc: 0.6131
Epoch 6/10
110288/110288 [==============================] - 6s 53us/step - loss: 1.3657 - acc: 0.6199 - val_loss: 1.3498 - val_acc: 0.6241
Epoch 7/10
110288/110288 [==============================] - 6s 53us/step - loss: 1.3259 - acc: 0.6263 - val_loss: 1

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [17]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    # TODO: Train the final model
    #print("Debug input_shape =" , input_shape, "  input_shape[1:] =", input_shape[1:], "  Input length=", input_shape[1:][0])
    #print("Debug output_sequence_length =" , output_sequence_length) 
    #print("Debug english_vocab_size =" , english_vocab_size)
    #print("Debug french_vocab_size =" , french_vocab_size)
    
    
    ## Model_final is a seq2seq model (encoder-decoder) using embedding and bidirectional LSTM
    
    # Hyperparameters
    embedding_size = 128
    rnn_cells = 300
    dropout = 0.2
    learning_rate = 1e-3
    
    from keras.layers import LSTM, concatenate
    
    # Input and embedding
    encoder_input_seq = Input(shape=input_shape[1:]) 
    embedded_input_seq = Embedding(input_dim = english_vocab_size,
                                   output_dim = embedding_size,
                                   input_length=input_shape[1:][0])(encoder_input_seq)
    
    ## Note Bidirectional LSTM is used on the encoder side (see https://arxiv.org/pdf/1609.08144.pdf )
    ## Alternate version : Encoder RNN Bidirectional layer (Using the Keras Bidirectional layer wrappers)
    #encoder_output, forward_state_h, forward_state_c, backward_state_h, backward_state_c = Bidirectional(LSTM(units=rnn_cells,
    #                                                                                                          dropout=dropout,
    #                                                                                                          return_sequences=False,
    #                                                                                                          return_state=True))(embedded_input_seq)
    
    # Encoder Forward RNN layer
    encoder_forward_output, forward_state_h, forward_state_c = LSTM(units=rnn_cells,
                                                                    dropout=dropout,
                                                                    return_sequences=False,
                                                                    return_state=True,
                                                                    go_backwards=False)(embedded_input_seq)
    
    # Encoder backward RNN layer
    encoder_backward_output, backward_state_h, backward_state_c = LSTM(units=rnn_cells,
                                                                       dropout=dropout,
                                                                       return_sequences=False,
                                                                       return_state=True,
                                                                       go_backwards=True)(embedded_input_seq)
    
    # Encoder output and states : Merge the LSTM Forward and Backward ouputs (using 'concatenate' method) 
    state_h = concatenate([forward_state_h, backward_state_h]) 
    state_c = concatenate([forward_state_c, backward_state_c])    
    encoder_output = concatenate([encoder_forward_output, encoder_backward_output])        
         
    # Decoder Input   
    decoder_input_seq = RepeatVector(output_sequence_length)(encoder_output)
    
    # Decoder RNN layer
    # Note : we need twice more LSTM cells as we have concatenated backward and forward LSTM layers in encoder
    decoder_output = LSTM(units=rnn_cells*2,
                                  dropout=dropout,
                                  return_sequences=True,
                                  return_state=False,
                                  go_backwards=False)(decoder_input_seq, initial_state=[state_h, state_c])
    
    # Decoder output
    logits = TimeDistributed(Dense(units=french_vocab_size))(decoder_output) 
    
    # Model
    model = Model(encoder_input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])
    
    return model    
       
tests.test_model_final(model_final)
print('Final Model Loaded\n')

# Train and Print prediction(s)

# Pad the input to work with the Embedding layer
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
#print("Debug tmp_x shape=", tmp_x.shape )


# Train the neural network 
final_rnn_model = model_final(input_shape = tmp_x.shape,
                              output_sequence_length = max_french_sequence_length,
                              english_vocab_size = english_vocab_size+1,
                              french_vocab_size = french_vocab_size+1)

print(final_rnn_model.summary(line_length=125))


final_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(final_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))
    

Final Model Loaded

_____________________________________________________________________________________________________________________________
Layer (type)                             Output Shape               Param #        Connected to                              
input_8 (InputLayer)                     (None, 21)                 0                                                        
_____________________________________________________________________________________________________________________________
embedding_4 (Embedding)                  (None, 21, 128)            25600          input_8[0][0]                             
_____________________________________________________________________________________________________________________________
lstm_4 (LSTM)                            [(None, 300), (None, 300), 514800         embedding_4[0][0]                         
__________________________________________________________________________________________________

## Prediction (IMPLEMENTATION)

In [18]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = model_final(input_shape = x.shape,
                        output_sequence_length = y.shape[1],
                        english_vocab_size = len(x_tk.word_index)+1,
                        french_vocab_size = len(y_tk.word_index)+1)

    model.fit(x, y, batch_size=1024, epochs=20, validation_split=0.2)
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))

final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 19s 174us/step - loss: 2.6454 - acc: 0.4714 - val_loss: 1.7391 - val_acc: 0.5637
Epoch 2/20
110288/110288 [==============================] - 17s 154us/step - loss: 1.4921 - acc: 0.6008 - val_loss: 1.2942 - val_acc: 0.6438
Epoch 3/20
110288/110288 [==============================] - 17s 154us/step - loss: 1.2154 - acc: 0.6622 - val_loss: 1.1042 - val_acc: 0.6922
Epoch 4/20
110288/110288 [==============================] - 17s 154us/step - loss: 1.0529 - acc: 0.6982 - val_loss: 0.9659 - val_acc: 0.7170
Epoch 5/20
110288/110288 [==============================] - 17s 155us/step - loss: 0.9242 - acc: 0.7263 - val_loss: 0.8589 - val_acc: 0.7407
Epoch 6/20
110288/110288 [==============================] - 17s 154us/step - loss: 0.8348 - acc: 0.7470 - val_loss: 0.7325 - val_acc: 0.7761
Epoch 7/20
110288/110288 [==============================] - 17s 155us/step - loss: 0.7114 - acc: 0.7803

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [ ]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?